In [ ]:
import os
from dotenv import load_dotenv
from typing import TypedDict

load_dotenv()


True

In [ ]:
from langgraph.graph import StateGraph, START, END


In [ ]:
class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int
    sr: float # strike rate
    bpb: float # balls per boundary
    bp: float # boundary percent
    summary: str


In [ ]:
def calculate_sr(state: BatsmanState) -> BatsmanState:
    runs = state['runs']
    balls = state['balls']
    strike_rate = (runs/balls) * 100
    
    return {'sr': strike_rate}


In [ ]:
def calculate_bpb(state: BatsmanState) -> BatsmanState:
    balls = state['balls']
    fours = state['fours']
    sixes = state['sixes']
    balls_per_boundary = balls/(fours + sixes)
    
    return {'bpb': balls_per_boundary}


In [ ]:
def calculate_bp(state: BatsmanState) -> BatsmanState:
    fours = state['fours']
    sixes = state['sixes']
    runs = state['runs']
    total_boundary_run = (fours * 4) + (sixes * 6)
    boundary_percent = (total_boundary_run/runs) * 100
    
    return {'bp': round(boundary_percent, 2)}


In [ ]:
def generate_summary(state: BatsmanState):
    summary = f"""
    Strike Rate - {state['sr']}
    Balls Per Boundary - {state['bpb']}
    Boundary Percent - {state['bp']}%
    """
    
    return {'summary': summary}


In [ ]:
graph = StateGraph(BatsmanState);

graph.add_node('calculate_sr', calculate_sr)
graph.add_node('calculate_bpb', calculate_bpb)
graph.add_node('calculate_bp', calculate_bp)
graph.add_node('generate_summary', generate_summary)

graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_bp')
graph.add_edge('calculate_sr', 'generate_summary')
graph.add_edge('calculate_bpb', 'generate_summary')
graph.add_edge('calculate_bp', 'generate_summary')
graph.add_edge('generate_summary', END)

workflow = graph.compile()


In [ ]:
initial_state = {
    'runs': 108,
    'balls': 50,
    'fours': 6,
    'sixes': 4,
}

final_state = workflow.invoke(initial_state)


In [ ]:
print(final_state['summary'])



    Strike Rate - 216.0
    Balls Per Boundary - 5.0
    Boundary Percent - 44.44%
    
